# 导入

In [16]:
import numpy as np
from bertopic import BERTopic
from transformers.pipelines import pipeline
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

# 加载文件

In [17]:

with open('./data/切词.txt', 'r', encoding='utf-8') as file:
  docs = file.readlines()
print('条数: ', len(docs))
print('预览第一条: ', docs[0])

vectorizer_model = None

条数:  1000
预览第一条:  文旅文 创看 洛阳 河南省 文旅文创 发展 大会 本次 大会 安排 项目 签约 主要 方面 内容 一是 文旅 产业 项目 签约 截至 目前 梳理 重点 文旅 项目 投资总额 525.6 亿元 遴选 重大项目 进行 现场 签约 投资总额 365.8 亿元 项目 包括 文物 数字化 开发 文化 创意 园区 建设 文化 项目 涵盖 旅游 度假区 建设 旅游 酒店 民宿 打造 旅游 项目 既有 旅游 景区 开发 商旅 综合体 建设 传统 业态 项目 宇宙 基地 沉浸 演艺 业态 项目 充分体现 我省 文化 旅游 发展 特点 趋势 二是 引客 入豫 项目 签约 主要 我省 文旅 部门 文旅 企业 头部 旅行 知名 OTA 平台 重点 客源地 文旅 部门 签订 引客 入豫 协议 持续 拓展 省外 客源 市场



# 创建模型

In [18]:
# 1. 词向量模型，同时加载本地训练好的词向量
embedding_model = pipeline("feature-extraction", model="bert-base-chinese") # 使用bert-base-chinese
embeddings = np.load('./data/embedding_bbc.npy') # 使用bert-base-chinese向量
print('向量shape：', embeddings.shape)

# 替换: 使用hfl模型
# embedding_model = pipeline("feature-extraction", model="hfl/chinese-bert-wwm")
# embeddings = np.load('./data/embedding_hfl.npy') 
# print('向量shape：', embeddings.shape)

# 替换: 使用Sentencetransformers模型
# embedding_model = embedding_model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2',)
# embeddings = np.load('./data/embedding_sen.npy')
# print(embeddings.shape)

# 2. 创建UMAP降维模型
umap_model = UMAP(
  n_neighbors=15,
  n_components=5,
  min_dist=0.0,
  metric='cosine',
  random_state=30  # ⚠️ 防止随机 https://maartengr.github.io/BERTopic/faq.html
)

# 3. 创建HDBSCAN聚类模型
# 如果要建设离群值，可以减小下面两个参数min_cluster_size min_samples
# https://hdbscan.readthedocs.io/en/latest/faq.html
hdbscan_model = HDBSCAN(
  min_cluster_size=10,
  min_samples=5,
  metric='euclidean'
)

# 5. 创建CountVectorizer模型
vectorizer_model = CountVectorizer(stop_words=['洛阳', '旅游', '文化'])

# 6. 正式创建BERTopic模型
topic_model = BERTopic(
  embedding_model=embedding_model,
  vectorizer_model=vectorizer_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


向量shape： (1000, 768)


# 训练模型

In [19]:
# 查看主题
topics, probs = topic_model.fit_transform(docs, embeddings=embeddings) #传入训练好的词向量
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,335,-1_景区_城市_河南_中国,"[景区, 城市, 河南, 中国, 游客, 活动, 洛阳市, 博物馆, 遗址, 栾川]",[河南 多家 景区 陆续 发布 开园 公告 台风 杜苏芮 强度 逐渐 减弱 河南 景区 陆续...
1,0,60,0_活动_门票_景区_免费,"[活动, 门票, 景区, 免费, 时间, 高速, 栾川, 地图, 白云山, 游客]",[洛阳 身边 自驾游 栾川 高速 免费 答疑 自驾游 栾川 高速 费全免 问题 需要 了解 ...
2,1,58,1_王府_竹海_旅行_河南,"[王府, 竹海, 旅行, 河南, 景区, 仙门, 旅游区, 中国, 三彩, 栾川]",[旅行 推荐 旅行 冬天 就要 穿过 中国 zui 天门 广东 zui 早古 驿道 仙气 飘...
3,2,50,2_石窟_中国_龙门石窟_艺术,"[石窟, 中国, 龙门石窟, 艺术, 世界, 文化遗产, 少林寺, 朝代, 造像, 石刻]",[洛阳 周边游 龙门石窟 中国 石刻 艺术 宝库 现为 世界 文化遗产 全国 重点 文物保护...
4,3,47,3_旅客_年票_客流_郑州,"[旅客, 年票, 客流, 郑州, 列车, 客运, 发送, 出行, 公交, 高速]",[今天 郑州 铁路 预计 发送 旅客 51.7 万人 管内 客流量 激增 中国 铁路 郑州 ...
5,4,43,4_机场_建设_规划_项目,"[机场, 建设, 规划, 项目, 自驾车, 高铁, 铁路, 航线, 黄河, 国家]",[国家 发展 改革 印发 汉江 生态 经济带 发展 规划 通知 汉江 生态 经济带 规划 范...
6,5,42,5_地方_很多_一天_一点,"[地方, 很多, 一天, 一点, 已经, 时间, 古城, 酒店, 西安, 感受]",[洛阳 值得 一去 地方 简单 下来 洛阳 旅游 攻略 建议 夏天 洛阳 洛阳 市区 主要 ...
7,6,41,6_泉州_洛阳桥_旅行_历史,"[泉州, 洛阳桥, 旅行, 历史, 济南, 中国, 黄陂, 沧州, 古城, 洛邑]",[海丝 泉州 夏天 清晨 洛阳桥 洛阳 日出 夏天 清晨 来到 泉州 洛阳桥 沐浴 晨光 长...
8,7,34,7_工作_集团_河洛_建设,"[工作, 集团, 河洛, 建设, 投资, 项目, 发展, 大鼓, 服务, 进行]",[地方 国资 洛阳市 国资委 检查组 检查 督导 洛阳 文化 旅游 投资 集团 国企改革 三...
9,8,32,8_游客_景区_午餐_老君山,"[游客, 景区, 午餐, 老君山, 网友, 河南, 表示, 同学, 卷入, 孙军范]",[无人 值守 一元 午餐 结账 多出 国庆 假期 景区 连续 推出 午餐 共售 游客 点赞 ...


# 保存聚类结果

In [20]:
topic_docs = topic_model.get_document_info(docs)
topic_docs.to_csv('./聚类结果.csv')

In [21]:
with open('./data/文本.txt', 'r', encoding='utf-8') as file:
  texts = file.readlines()
  print('文本条数：', len(texts))
  topic_docs.insert(1, '原文', texts)
with open('./data/时间.txt', 'r', encoding='utf-8') as file:
  years = file.readlines()
  print('文本条数：', len(years))
  topic_docs.insert(2, '时间', years)
topic_docs.to_csv('./聚类结果2.csv')

文本条数： 1000
文本条数： 1000


# 可视化

In [22]:
topic_model.visualize_barchart()

In [23]:
topic_model.visualize_topics()

In [24]:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, hide_document_hover=True)

# 层次聚类

In [25]:
hierarchical_topics = topic_model.hierarchical_topics(docs)
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

100%|██████████| 20/20 [00:00<00:00, 231.61it/s]


# 合并主题

In [26]:
topic_model.merge_topics(docs, [
 [19, 2],
 [9, 6, 1, 12, 20, 5, 15, 18],
 [11, 16, 4, 10, 7],
 [17, 13, 0, 3, 8, 14]
])

topic_info = topic_model.get_topic_info()
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,335,-1_景区_河南_城市_中国,"[景区, 河南, 城市, 中国, 游客, 活动, 洛阳市, 博物馆, 遗址, 发展]",[河南 多家 景区 陆续 发布 开园 公告 台风 杜苏芮 强度 逐渐 减弱 河南 景区 陆续...
1,0,244,0_河南_旅行_历史_中国,"[河南, 旅行, 历史, 中国, 景区, 地方, 古城, 位于, 泉州, 郑州]",[文旅 河南 Hello 世界 外国 嘉宾 河南 印象 河南 历史 文化 圈粉 无数 日本 ...
2,1,205,1_景区_游客_郑州_旅客,"[景区, 游客, 郑州, 旅客, 活动, 门票, 年票, 高速, 假期, 时间]",[洛阳 热门 栾川县 八大 景区 洛阳 市民 门票 第七个 中国 旅游 栾川县 八大 景区 ...
3,2,154,2_建设_发展_项目_河南,"[建设, 发展, 项目, 河南, 博物馆, 国家, 工作, 文旅, 洛阳市, 城市]",[行走 河南 读懂 中国 关注 全省 文旅文创 发展 大会 二十大 报告 指出 坚持 以文塑...
4,3,62,3_中国_龙门石窟_石窟_艺术,"[中国, 龙门石窟, 石窟, 艺术, 世界, 文化遗产, 少林寺, 朝代, 造像, 位于]",[洛阳 周边游 龙门石窟 中国 石刻 艺术 宝库 现为 世界 文化遗产 全国 重点 文物保护...


In [27]:
topic_model.visualize_topics()

In [28]:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, hide_document_hover=True)

# 动态主题模型

In [29]:
# 读取时间戳
with open('./data/时间.txt', "r", encoding='utf-8') as file:
    lines = file.readlines()
    timestamps = [int(line.strip()) for line in lines]
print(len(timestamps), timestamps[:10])

1000 [2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023]


In [30]:
topics_over_time = topic_model.topics_over_time(docs, timestamps, global_tuning=False, evolution_tuning=False)
topic_model.visualize_topics_over_time(topics_over_time)